<a href="https://colab.research.google.com/github/marquezjp/SQLToPandas/blob/main/Oracle/Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exportar uma Query do Oracle para um DataFrame

## Preparar Ambiente

Para instalar o [cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)

[Download](https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html) do [Oracle Instant Client Basic Light](https://download.oracle.com/otn_software/nt/instantclient/19900/instantclient-basiclite-windows.x64-19.9.0.0.0dbru.zip)

```
!pip install cx_Oracle --upgrade
```

```
cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient")
```

Para instalar o XlsxWriter

```
!pip install -U XlsxWriter
```

In [ ]:
# Carrega a Biblioteca do Oracle, YAML e Pandas
from datetime import date
import csv
import pandas as pd

In [ ]:
import xlsxwriter

In [ ]:
import sqlite3
import cx_Oracle
from sqlalchemy.engine import create_engine

In [ ]:
today = date.today().strftime("%d/%m/%Y")
print("Hoje é", today)

Hoje é 22/12/2020


In [ ]:
!dir

## Conexão via SQL Alchemy

In [ ]:
# Cria a Conexão do Oracle
oracleConfig = {
    'dialect': 'oracle',
    'sql_driver': 'cx_oracle',
    'dsn_tns': {
        'host': '192.168.10.247',
        'service': 'sigrhprod.financas.pref',
        'port': 1521},
    'user': 'sigrhprd',
    'password': 'prdM4c3i0'
}

dialeto = oracleConfig.get('dialect', 'oracle')
sql_driver = oracleConfig.get('sql_driver', 'cx_oracle')

dsn_tnsConfig = oracleConfig.get('dsn_tns', {'host': '', 'service': '', 'port': 1521})
dsn_tns = cx_Oracle.makedsn(dsn_tnsConfig.get('host', '0.0.0.0'),
                            dsn_tnsConfig.get('port', '1521'),
                            service_name=dsn_tnsConfig.get('service', 'orcl'))

user=oracleConfig.get('user', '')
password=oracleConfig.get('password', '')

conexaoOracle = create_engine(f'{dialeto}+{sql_driver}://{user}:{password}@{dsn_tns}')
conexaoOracle

Engine(oracle+cx_oracle://sigrhprd:***@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=192.168.10.247)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=sigrhprod.financas.pref))))

In [ ]:
# Cria a Conexão do SQLite
sqliteConfig = {
    'dialect': 'sqlite',
    'dbworkpath': 'SQLite',
    'dbfilename': 'contracheque',
    'dbext': 'sqlite'
}

dialeto = sqliteConfig.get('dialect', 'sqlite')

dbworkpath = sqliteConfig.get('dbworkpath', '')
dbfilename = sqliteConfig.get('dbfilename', '')
dbext = sqliteConfig.get('dbext', '')
dburl = f'/{dbworkpath}/{dbfilename}.{dbext}' if dbfilename else ''

conexaoSQLite = create_engine(f'{dialeto}://{dburl}', echo=False)
conexaoSQLite

Engine(sqlite:///SQLite/contracheque.sqlite)

## Consulta SQL

### Consulta dos Campos das Tabelas do Modelo de Dados do SIGRH

In [ ]:
# Consulta dos Campos das Tabelas do Modelo de Dados do SIGRH
SQL = '''
select lower(substr(table_name,1,4)) as SubSystem, lower(table_name) as TableName, lower(column_name) as ColumnName
from user_tab_columns
'''

### Consulta Servidores Ativos em uma Referencia

In [ ]:
# Consulta Servidores Ativos em uma Referencia
Referencia = '202003'
dataInicioReferencia = f'01-{Referencia[4:6]}-{Referencia[0:4]}'
SQL = f'''
select  '{Referencia}' as MesAno,
        o.sgorgao as OrgaoFolha,
        p.nmpessoa as Nome,
        lpad(v.numatricula, 7, 0)||'-'||v.nudvmatricula as Matricula,
        p.nucpf as CPF

from ecadvinculo v
inner join ecadpessoa p on p.cdpessoa = v.cdpessoa
inner join vcadorgao o on o.cdorgao = v.cdorgao 
inner join (
    select p.nucpf as nucpf, count(*) as vinculos
    from ecadvinculo v
    inner join ecadpessoa p on p.cdpessoa = v.cdpessoa
    where v.flanulado = 'N'
      and (v.dtadmissao < last_day('{dataInicioReferencia}') + 1)
      and (v.dtdesligamento is null or v.dtdesligamento > last_day('{dataInicioReferencia}'))
    group by p.nucpf
    having count(*) > 1
) d on d.nucpf = p.nucpf
order by p.nucpf
'''

### Consulta Dados Funcionais dos Servidores

In [ ]:
# Consulta Dados Funcionais dos Servidores
SQL = '''
select o.sgorgao as Orgao,
	   lpad(p.nucpf, 11, 0) CPF,
       lpad(v.numatricula || '-' || nudvmatricula,9,0) as Matricula,
       p.nmpessoa as Nome,
       p.dtnascimento as dtNascimento,
	   trunc((sysdate - p.dtnascimento) / 365) as Idade,
	   p.flsexo as Sexo,
       v.dtadmissao as dtAdmissao,
	   v.dtdesligamento as dtDesligamento,
       
       --c.deitemcarreira as Cargo,
       cr.deitemcarreira as Carreira,
       c.deitemcarreira as Cargo,
	   nr.nugruposalarial || e.nunivelpagamento || e.nureferenciapagamento as NivelAtual,

       rt.nmrelacaotrabalho as RelacaoTrabalho,
       rgtb.nmregimetrabalho as RegimeTrabalho,
       rgpv.nmregimeprevidenciario as RegimePrevidenciario,

	   d.decargocomissionado as CargoComissionado,
       ecc.nureferencia || ecc.nunivel as NivelComissionado,
       
       
       case
            when (select count(*) from ecadhistcargoefetivo cef 
                  where cef.cdvinculo = v.cdvinculo and cef.cdrelacaotrabalho = 5
                  and cef.flanulado = 'N'
				  and cef.dtinicio < last_day(sysdate) + 1
				  and (cef.dtfim is null or cef.dtfim > last_day(sysdate))
                  and not exists (select 1 from ecadhistcargocom cco
                                   where cco.cdvinculo = v.cdvinculo
                                     and cco.flanulado = 'N'
									 and cco.dtinicio < last_day(sysdate) + 1
									 and (cco.dtfim is null or cco.dtfim > last_day(sysdate)))
            ) > 0 then 'EFETIVO'
            when (select count(*) from ecadhistcargoefetivo cef 
                   where cef.cdvinculo = v.cdvinculo and cef.cdrelacaotrabalho = 5
                     and cef.flanulado = 'N'
					 and cef.dtinicio < last_day(sysdate) + 1
					 and (cef.dtfim is null or cef.dtfim > last_day(sysdate)) 
                     and exists(select 1 from ecadhistcargocom cco
                                 where cco.cdvinculo = v.cdvinculo
                                   and cco.flanulado = 'N'
								   and cco.dtinicio < last_day(sysdate) + 1
								   and (cco.dtfim is null or cco.dtfim > last_day(sysdate)))
            ) > 0 then 'EFETIVO + COMISSIONADO'
            when (select count(*) from ecadhistcargoefetivo cef 
                    where cef.cdvinculo = v.cdvinculo and cef.cdrelacaotrabalho = 10
                      and cef.flanulado = 'N'
					  and cef.dtinicio < last_day(sysdate) + 1
					  and (cef.dtfim is null or cef.dtfim > last_day(sysdate))
                      and not exists (select 1 from ecadhistcargocom cco
                                       where cco.cdvinculo = v.cdvinculo
                                         and cco.flanulado = 'N'
										 and cco.dtinicio < last_day(sysdate) + 1
										 and (cco.dtfim is null or cco.dtfim > last_day(sysdate)))
            ) > 0 then 'DISPOSIÇÃO'
            when (select count(*) from ecadhistcargoefetivo cef 
                   where cef.cdvinculo = v.cdvinculo and cef.cdrelacaotrabalho = 10
                     and cef.flanulado = 'N'
					 and cef.dtinicio < last_day(sysdate) + 1
					 and (cef.dtfim is null or cef.dtfim > last_day(sysdate))	  
                     and exists(select 1 from ecadhistcargocom cco
                                 where cco.cdvinculo = v.cdvinculo
                                   and cco.flanulado = 'N'
								   and cco.dtinicio < last_day(sysdate) + 1
								   and (cco.dtfim is null or cco.dtfim > last_day(sysdate)))
            ) > 0 then 'DISPOSIÇÃO + COMISSIONADO'
            when (select count(*) from ecadhistcargocom cco
                   where cco.cdvinculo = v.cdvinculo 
                     and cco.flanulado = 'N'
					 and cco.dtinicio < last_day(sysdate) + 1
					 and (cco.dtfim is null or cco.dtfim > last_day(sysdate))
            ) > 0 then 'COMISSIONADO PURO'
            when (select count(*) from ecadhistfuncaochefia fuc 
                   where fuc.cdvinculo = v.cdvinculo 
                     and fuc.flanulado = 'N'
					 and fuc.dtinicio < last_day(sysdate) + 1
					 and (fuc.dtfim is null or fuc.dtfim > last_day(sysdate)) 
                     and not exists (select 1 from ecadhistcargoefetivo cef
                                      where cef.cdvinculo = v.cdvinculo
                                        and cef.flanulado = 'N'
										and cef.dtinicio < last_day(sysdate) + 1
										and (cef.dtfim is null or cef.dtfim > last_day(sysdate)))
                     and not exists (select 1 from ecadhistcargocom cco
                                      where cco.cdvinculo = v.cdvinculo
                                        and cco.flanulado = 'N'
										and cco.dtinicio < last_day(sysdate) + 1
										and (cco.dtfim is null or cco.dtfim > last_day(sysdate)))
                     and not exists (select 1 from epvdconcessaoaposentadoria apo
                                      where apo.cdvinculo = v.cdvinculo
                                        and apo.flanulado = 'N' and apo.flativa = 'S'
										and apo.dtinicioaposentadoria < last_day(sysdate) + 1
										and (apo.dtfimaposentadoria is null or apo.dtfimaposentadoria > last_day(sysdate)))
            ) > 0 then 'APENAS FUNCAO GRATIFICADA'
            when (select count(*) from ecadhistcargoefetivo cef
                    where cef.cdvinculo = v.cdvinculo and cef.cdrelacaotrabalho = 3
                      and cef.flanulado = 'N'
					  and cef.dtinicio < last_day(sysdate) + 1
					  and (cef.dtfim is null or cef.dtfim > last_day(sysdate))
            ) > 0 then 'ACT' 
            when (select count(*) from ecadhistestagio est
                   where est.cdvinculoestagio = v.cdvinculo 
                     and est.flanulado = 'N'
					 and est.dtinicio < last_day(sysdate) + 1
					 and (est.dtfim is null or est.dtfim > last_day(sysdate))
            ) > 0 then 'ESTAGIÁRIO'
            when (select count(*) from epvdconcessaoaposentadoria apo
                   where apo.cdvinculo = v.cdvinculo and apo.flativa = 'S'
                     and apo.flanulado = 'N'
					 and apo.dtinicioaposentadoria < last_day(sysdate) + 1
					 and (apo.dtfimaposentadoria is null or apo.dtfimaposentadoria > last_day(sysdate))
            ) > 0 then 'APOSENTADO'
            when (select count(*) from epvdhistpensaoprevidenciaria pen
                   where pen.cdvinculo = v.cdvinculo 
                     and pen.flanulado = 'N'
					 and pen.dtinicio < last_day(sysdate) + 1
					 and (pen.dtfim is null or pen.dtfim > last_day(sysdate))
            ) > 0 then 'PENSÃO PREVIDENCIÁRIA'
            when (select count(*) from epvdhistpensaonaoprev penesp
                   where penesp.cdvinculobeneficiario = v.cdvinculo 
                     and penesp.flanulado = 'N'
					 and penesp.dtinicio < last_day(sysdate) + 1
					 and (penesp.dtfim is null or penesp.dtfim > last_day(sysdate))
            ) > 0 then 'PENSÃO NÃO PREVIDENCIÁRIA' 
            else ' ' end Relacao,

       case when exists (select a.cdvinculo from eafaafastamentovinculo a
                         left join eafamotivoafasttemporario t on t.cdmotivoafasttemporario = a.cdmotivoafasttemporario
                         left join eafahistmotivoafasttemp ht on ht.cdmotivoafasttemporario = t.cdmotivoafasttemporario
                         where a.flanulado = 'N'
                           and a.dtinicio < last_day(sysdate) + 1
                           and (a.dtfim is null or a.dtfim > last_day(sysdate))
                           and (a.fltipoafastamento = 'D' or (a.fltipoafastamento = 'T' and ht.flremunerado = 'N'))
                           and a.cdvinculo = v.cdvinculo
                         )
            then 'SIM' else 'NAO' end as Afastado

from ecadvinculo v
inner join vcadorgao o on o.cdorgao = v.cdorgao 
inner join ecadpessoa p on p.cdpessoa = v.cdpessoa
 
left join ecadhistcargoefetivo e on e.cdvinculo = v.cdvinculo
      and (e.dtinicio < last_day(sysdate) + 1) and (e.dtfim is null or e.dtfim > last_day(sysdate))
left join ecadhistnivelrefcef nr on nr.cdhistcargoefetivo = e.cdhistcargoefetivo
      and (nr.dtinicio < last_day(sysdate) + 1) and (nr.dtfim is null or nr.dtfim > last_day(sysdate))
left join ecadestruturacarreira es on es.cdestruturacarreira = e.cdestruturacarreira
left join ecaditemcarreira c on c.cditemcarreira = es.cditemcarreira
left join ecadestruturacarreira cp on cp.cdestruturacarreira = es.cdestruturacarreirapai
left join ecaditemcarreira cr on cr.cditemcarreira = cp.cditemcarreira
left join ecadrelacaotrabalho rt on rt.cdrelacaotrabalho = e.cdrelacaotrabalho
left join ecadregimetrabalho rgtb on rgtb.cdregimetrabalho = v.cdregimetrabalho
left join ecadregimeprevidenciario rgpv on rgpv.cdregimeprevidenciario = v.cdregimeprevidenciario
 
left join ecadhistcargocom ecc on ecc.cdvinculo = v.cdvinculo
      and (ecc.dtinicio < last_day(sysdate) + 1) and (ecc.dtfim is null or ecc.dtfim > last_day(sysdate))
left join ecadcargocomissionado cco on cco.cdcargocomissionado = ecc.cdcargocomissionado
left join ecadevolucaocargocomissionado d on d.cdcargocomissionado = ecc.cdcargocomissionado
      and (d.dtiniciovigencia < last_day(sysdate) + 1) and (d.dtfimvigencia is null or d.dtfimvigencia > last_day(sysdate))
left join ecadlocaltrabalho lt on lt.cdhistcargocom = ecc.cdhistcargocom
      and (lt.dtinicio < last_day(sysdate) + 1) and (lt.dtfim is null or lt.dtfim > last_day(sysdate))
left join vcadunidadeorganizacional u on u.cdunidadeorganizacional = lt.cdunidadeorganizacional
      and (u.dtiniciovigencia < last_day(sysdate) + 1) and (u.dtfimvigencia is null or u.dtfimvigencia > last_day(sysdate))

where v.flanulado = 'N' --and (v.dtdesligamento is null or v.dtdesligamento > last_day(sysdate))
  --and v.cdvinculo not in (select v.cdvinculo from ecadvinculo v
  --                        inner join vcadorgao o on o.cdorgao = v.cdorgao
  --                        where v.dtadmissao < last_day(sysdate) + 1
  --                          and (v.dtdesligamento > last_day(sysdate) or v.dtdesligamento is null)
  --                          and o.sgorgao != 'COMARHP'
  --                          and v.cdpessoa in (select v.cdpessoa from ecadvinculo v
  --                                             inner join ecadpessoa p on p.cdpessoa = v.cdpessoa
  --                                             inner join vcadorgao o on o.cdorgao = v.cdorgao
  --                                             where v.dtadmissao < last_day(sysdate) + 1
  --                                               and (v.dtdesligamento > last_day(sysdate) or v.dtdesligamento is null)
  --                                               and o.sgorgao = 'COMARHP'))
'''

### Consulta dos Totais das Folhas do SIGRH

In [ ]:
# Consulta dos Totais das Folhas do SIGRH
SQL = '''
select 
     f.nuanomesreferencia     AnoMes,
     f.nuanoreferencia        Ano,
     f.numesreferencia        Mes,
     o.cdorgaosirh            Codigo,
     o.sgorgao                Orgao,
     tfo.nmtipofolhapagamento Folha,
     tc.nmtipocalculo         Tipo,
     sum(nvl(capa.vlproventos, 0))   Proventos, -- Having 8 > 0
     sum(nvl(capa.vldescontos, 0))   Descontos,
     sum(nvl(capa.vlproventos, 0) - nvl(capa.vldescontos, 0)) Liquido,
     count(*)                 Servidores
  
from epagcapahistrubricavinculo capa
inner join epagfolhapagamento f on f.cdfolhapagamento = capa.cdfolhapagamento
inner join epagtipofolhapagamento tfo on tfo.cdtipofolhapagamento = f.cdtipofolhapagamento
inner join epagtipocalculo tc on tc.cdtipocalculo = f.cdtipocalculo
inner join vcadorgao o on o.cdorgao = f.cdorgao
          
where f.flcalculodefinitivo = 'S'
  --and f.nuanomesreferencia = 202011
  --and f.nuanoreferencia in (2020, 2019, 2018)
  --and f.numesreferencia in (09, 10, 11)
  --and  o.sgorgao = 'SEMGE'
          
group by f.nuanomesreferencia, f.nuanoreferencia, f.numesreferencia, tfo.nmtipofolhapagamento, tc.nmtipocalculo, o.cdorgaosirh, o.sgorgao
having  sum(nvl(capa.vlproventos, 0)) > 0
order by f.nuanomesreferencia, f.nuanoreferencia, f.numesreferencia, o.cdorgaosirh, tfo.nmtipofolhapagamento, tc.nmtipocalculo
'''

### Consulta dos ContraCheques do SIGRH

In [ ]:
# Consulta dos ContraCheques do SIGRH
SQL = '''
select 
 o.sgorgao as Orgao,
 f.nuanoreferencia as Ano,
 f.numesreferencia as Mes,
 tf.nmtipofolhapagamento as Folha,
 case f.cdtipocalculo
  when 1 then 'NORMAL'
  when 5 then 'SUPLEMENTAR'
  else to_char(f.cdtipocalculo)
 end as Calculo,
 lpad(f.nusequencialfolha,2,'0') as SeqFolha,
 case f.flcalculodefinitivo when 'N' then 'NÃO' when 'S' then 'SIM' else '' end as Definitivo,

 lpad(v.numatricula, 7, 0)||'-'||v.nudvmatricula as Matricula,
 lpad(p.nucpf, 11, 0) as CPF,
 p.nmpessoa as Nome,

 Case capa.flativo
  when 'S' then 'ATIVO'
  when 'N' then 'INATIVO'
  else ' '
 end Situacao,
 rtr.nmregimetrabalho as RegimeTrabalho,
 rt.nmrelacaotrabalho as RelacaoTrabalho,
 rp.nmregimeprevidenciario RegimePrevidenciario,
 tr.nmtiporegimeproprioprev RegimePrevidenciarioProprio, 

 d.decargocomissionado as CargoComissionado,
 c.deitemcarreira as CargoEfetivo,

 hu.nmunidadeorganizacional as UnidadeOrganizacional,
 cc.nucentrocusto CodigoCentroCusto,
 cc.nmcentrocusto CentroCusto,

 case capa.sgtipocredito
  when 'FI' then 'Fundo Financeiro'
  when 'PR' then 'Fundo Previdenciario'
  when 'GE' then 'Geral - Comissionados'
  when 'GO' then 'Geral - CLT/Outros'
  else ' '
 end Fundo,

 case
  when pp.cdhistpensaoprevidenciaria is not null then 'PENSÃO PREVIDENCIÁRIA'
  when pnp.cdhistpensaonaoprev is not null then 'PENSÃO NÃO PREVIDENCIÁRIA'
  when capa.flativo = 'N' then 'INATIVO-APOSENTADO'
  when capa.cdregimetrabalho = 1 then 'CLT'
  when capa.cdrelacaotrabalho = 4 then 'AGENTE POLÍTICO'
  when cef.cdhistcargoefetivo is not null and capa.cdcargocomissionado is not null then 'EFETIVO + COMISSIONADO'  
  when capa.cdrelacaotrabalho = 3  then 'ACT'
  when capa.cdrelacaotrabalho = 5  then 'EFETIVO'
  when capa.cdrelacaotrabalho = 10 then 'EFETIVO À DISPOSICAO'
  when capa.cdrelacaotrabalho = 6  then 'COMISSIONADO'
  when cef.cdhistcargoefetivo is not null then 'EFETIVO' 
  else 'W-INDEFINIDO'
 end as Classificacao,

 case
  when rub.cdtiporubrica in (1, 2, 4, 10, 12) then '1-PROVENTO '
  when rub.cdtiporubrica in (5, 6, 8)         then '5-DESCONTO'
  when rub.cdtiporubrica = 9                  then '9-BASE DE CÁLCULO'
  else ' '
 end as GrupoRubrica,
 case rub.cdtiporubrica
  when  1 then '01-PROVENTO'
  when  2 then '02-DIF.PROVENTO'
  when  8 then '08-DEV.PROVENTO'
  when 10 then '10-EXFINDO.PROVENTO'
  when 12 then '12-EXFINDOANT.PROVENTO'
  when  5 then '05-DESCONTO'
  when  6 then '06-DIF.DESCONTO'
  when  4 then '04-DEV.DESCONTO'
  when  9 then '09-BASE'
  else ' '
 end as TipoRubrica,
 lpad(rub.nurubrica,4,'0') as Rubrica,
 lpad(pag.nusufixorubrica,2,'0') as Sufixo,
 rub.derubricaagrupamento as DeRubrica,
 case rub.flconsignacao when 'N' then 'NÃO' when 'S' then 'SIM' else '' end as Consignacao,
 ori.detipoorigemrubrica as Origem, 
 case when pag.cdexpressaoformcalc  is not null then 'SIM' 
      else 'NÃO'
 end as Tem_Formula,
 case when (lf.vlindice is null or lf.vlindice = 0) then 'NÃO'
      else 'SIM'
 end as Tem_Indice,
 case when (lf.vllancamentofinanceiro is null or lf.vllancamentofinanceiro = 0) then 'NÃO'
      else 'SIM'
 end as Valor_Informado,
 pag.nuparcela as Parcela,
 pag.vlindicerubrica as Indice,
 pag.vlpagamento as Valor

from epaghistoricorubricavinculo pag
inner join epagfolhapagamento f on f.cdfolhapagamento = pag.cdfolhapagamento and f.flcalculodefinitivo = 'S'
left join epagcapahistrubricavinculo capa on capa.cdfolhapagamento = pag.cdfolhapagamento and capa.cdvinculo = pag.cdvinculo
left join epagtipofolhapagamento tf on tf.cdtipofolhapagamento = f.cdtipofolhapagamento
left join vcadorgao o on o.cdorgao = f.cdorgao

left join ecadvinculo v on v.cdvinculo = pag.cdvinculo 
left join ecadpessoa p on p.cdpessoa = v.cdpessoa

left join ecadregimetrabalho rtr on rtr.cdregimetrabalho = capa.cdregimetrabalho
left join ecadrelacaotrabalho rt on rt.cdrelacaotrabalho = capa.cdrelacaotrabalho
left join ecadregimeprevidenciario rp on rp.cdregimeprevidenciario = v.cdregimeprevidenciario
left join ecadtiporegimeproprioprev tr on tr.cdtiporegimeproprioprev = v.cdtiporegimeproprioprev

left join ecadUnidadeOrganizacional u on u.cdunidadeorganizacional = capa.cdunidadeorganizacional
left join ecadHIstUnidadeOrganizacional hu on hu.cdUnidadeOrganizacional = u.cdUnidadeOrganizacional
       and hu.dtInicioVigencia <= last_day(sysdate) and (hu.Dtfimvigencia is null or hu.Dtfimvigencia >= last_day(sysdate))

left join ecadevolucaocargocomissionado d on d.cdcargocomissionado = capa.cdcargocomissionado
      and d.flanulado = 'N' and d.dtiniciovigencia <= last_day(sysdate) and (d.dtfimvigencia is null or d.dtfimvigencia >= last_day(sysdate))
left join ecadestruturacarreira es on es.cdestruturacarreira = capa.cdestruturacarreira
left join ecaditemcarreira c on c.cditemcarreira = es.cditemcarreira

left join ecadhistcargoefetivo cef on cef.cdvinculo = capa.cdvinculo and cef.flanulado = 'N' and cef.flprincipal = 'S'
  and cef.dtinicio <= last_day(sysdate) and (cef.dtfim is null or cef.dtfim >= last_day(sysdate))
left join epvdhistpensaoprevidenciaria pp on pp.cdvinculo = capa.cdvinculo and pp.flanulado = 'N'
left join epvdhistpensaonaoprev pnp on pnp.cdvinculobeneficiario = capa.cdvinculo and pnp.flanulado = 'N'

left join ecadcentrocusto cc on cc.cdcentrocusto = capa.cdcentrocusto

left join vpagrubricaagrupamento rub on rub.cdrubricaagrupamento = pag.cdrubricaagrupamento
left join epagtipoorigemrubrica ori on ori.cdtipoorigemrubrica = pag.cdtipoorigemrubrica
left join epaglancamentofinanceiro lf on lf.cdlancamentofinanceiro = pag.cdlancamentofinanceiro

--where f.nuanoreferencia = 2020
--  and o.sgorgao = 'SEMGE'

order by 2, 3, 1, 4, 5, 6, 7, 8
'''

### Consulta Resumo Rubrica por Ano, Mês, Folha

In [ ]:
# Consulta SQL
P_ANO = '2020'
P_MES = '10'
P_TIPO_FOLHA = 4 # 2-NORMAL  4-FERIAS  5-ESTAGIARIO  6-13 SAL

SQL = f'''
select Situacao,
       Fundo,
       Tipo,
       Rubrica,
       DeRubrica,
       Consignacao,
       Taxa,
       Orgao,
       CCusto,
       Sum(Montante) Montante,
       Sum(Valor_Retido) Valor_Retido

    from

    (
    select
        case when rub.cdtiporubrica = 5 then 'DESCONTO'
             when rub.cdtiporubrica = 6 then 'DIF.DESCONTO'
             when rub.cdtiporubrica = 4 then 'DEV.DESCONTO'
            else 'ERRO'
        end Tipo,
        rub.nurubrica Rubrica,
        rub.derubricaagrupamento DeRubrica,
        rub.flconsignacao Consignacao,
        hc.vltaxaretencao Taxa,
        o.nmorgao Orgao,
        cc.nucentrocusto||'-'||cc.nmcentrocusto CCusto,
        case when capa.sgtipocredito = 'FI' then 'Fundo Financeiro'
             when capa.sgtipocredito = 'PR' then 'Fundo Previdenciario'
             when capa.sgtipocredito = 'GE' then 'Geral - Comissionados'
             when capa.sgtipocredito = 'GO' then 'Geral - CLT/Outros'
            else ' '
        end Fundo,
        Case when capa.flativo = 'S' then 'ATIVO'
             when capa.flativo = 'N' then 'INATIVO'
            else ' '
        end Situacao,
        pag.vlpagamento Montante,
        trunc((hc.vltaxaretencao * pag.vlpagamento / 100), 2) Valor_Retido

     from epaghistoricorubricavinculo pag

        inner join epagfolhapagamento f
               on  f.cdfolhapagamento = pag.cdfolhapagamento
               and f.nuanoreferencia = {P_ANO}
               and f.numesreferencia = {P_MES}
               and f.cdtipofolhapagamento = {P_TIPO_FOLHA}
               and f.cdtipocalculo = 1

        inner join epagcapahistrubricavinculo capa on capa.cdvinculo = pag.cdvinculo and capa.cdfolhapagamento = pag.cdfolhapagamento
        inner join ecadcentrocusto cc on cc.cdcentrocusto = capa.cdcentrocusto
        inner join vcadorgao o on o.cdorgao = f.cdorgao
        inner join vpagrubricaagrupamento rub on rub.cdrubricaagrupamento = pag.cdrubricaagrupamento
        left  join epagconsignacao c on c.cdrubrica = rub.cdrubrica
        left  join epaghistconsignacao hc on hc.cdconsignacao = c.cdconsignacao and hc.dtfimvigencia is null

      where rub.cdtiporubrica in (5, 6, 4)

   )

      group by  Situacao, Fundo, Tipo, Rubrica, DeRubrica, Consignacao, Taxa, Orgao, CCusto
      order by  Situacao, Fundo, Tipo, Rubrica, DeRubrica, Consignacao, Taxa, Orgao, CCusto
'''

### Consulta de Servidores com mais de um Vinculo na Referencia

In [ ]:
# Consulta SQL
Referencia = '202012'
dataInicioReferencia = f'01-{Referencia[4:6]}-{Referencia[0:4]}'

SQL = f"""
select  '{Referencia}' as MesAno,
        o.sgorgao as OrgaoFolha,
        p.nmpessoa as Nome,
        lpad(v.numatricula, 7, 0)||'-'||v.nudvmatricula as Matricula,
        lpad(p.nucpf, 11, 0) as CPF

from ecadvinculo v
inner join ecadpessoa p on p.cdpessoa = v.cdpessoa
inner join vcadorgao o on o.cdorgao = v.cdorgao 
inner join (
    select p.nucpf as nucpf, count(*) as vinculos
    from ecadvinculo v
    inner join ecadpessoa p on p.cdpessoa = v.cdpessoa
    where v.flanulado = 'N'
      and (v.dtadmissao < last_day('{dataInicioReferencia}') + 1)
      and (v.dtdesligamento is null or v.dtdesligamento > last_day('{dataInicioReferencia}'))
    group by p.nucpf
    having count(*) > 1
) d on d.nucpf = p.nucpf
order by p.nucpf
"""

## Executar a Consulta

In [ ]:
%%timeit
base = pd.read_sql_query(SQL, conexaoOracle)
base.head()

In [ ]:
camposSelecionados = ['orgao', 'ano', 'mes', 'folha', 'calculo', 'matricula']
#print(base[camposSelecionados].iloc[0].values)
print(base[camposSelecionados].values[0])

## Salva os Dados Selecionados no Arquivo CSV

In [ ]:
%%timeit
nomeArquivo = 'sigrh_schema'
base.to_csv(f'{nomeArquivo}.csv', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
%%timeit
nomeArquivo = 'sigrh_schema'
compression_opts = dict(method='zip', archive_name=f'{nomeArquivo}.csv')
base.to_csv(f'{nomeArquivo}.zip', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC, compression=compression_opts)

## Salva os Dados Selecionados no Excel

In [ ]:
%%timeit
dataReferencia = date.today().strftime('%Y%m%d')
nomePlanilha = 'SCHEMA'
nomeArquivo = f'SIGRH-MCZ - {nomePlanilha} - {dataReferencia}.xlsx'

planilha = pd.ExcelWriter(nomeArquivo, engine='xlsxwriter', datetime_format='dd/mm/yy', date_format='dd/mm/yy')
base.to_excel(planilha, sheet_name=nomePlanilha, startrow=0, index=False)

# Formata Tabela
pagina = planilha.sheets[nomePlanilha]
linhas, colunas = base.shape
column_settings = [{'header': column} for column in base.columns.values.tolist()]
tabelaOpcoes = {
    'name': nomePlanilha,
    'style': 'Table Style Medium 11',
    'autofilter': False,
    'columns': column_settings
}
pagina.add_table(0, 0, linhas, colunas - 1, tabelaOpcoes)

# Formata Impressão
pagina.hide_gridlines(2)
pagina.freeze_panes(1, 0)
pagina.print_area(0,0,linhas, colunas)
pagina.repeat_rows(0,0)
pagina.fit_to_pages(1,0)
pagina.set_paper(9) # A4
pagina.set_portrait() # set_landscape or set_portrait()
#pagina.center_horizontally()
#pagina.center_vertically()
pagina.set_margins(left=0.7, right=0.7, top=1.14, bottom=0.75)

# Formata Cabecalho e Rodape
pagina.set_header('&R' + 'Data: &D')
pagina.set_footer('&R' +  'Página: &P of &N')

# Grava a Planilha
planilha.save()

In [ ]:
pd.read_excel(nomeArquivo, sheet_name=nomePlanilha, engine='openpyxl').head()

,subsystem,tablename,columnname
0,mlve,mlveventoversao,xmlesquema
1,acer,acerto_pen_alimento,codabate
2,acer,acerto_pen_alimento,codpensao
3,acer,acerto_pen_alimento,matricula
4,acer,acerto_pen_alimento_resumo,temredutor


## Salva os Dados Selecionados no Google Sheet

In [ ]:
# Carregar as Bibliotecas e Prover a Autenticação no Google Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
%%timeit
dataReferencia = date.today().strftime('%Y%m%d')
nomePlanilha = 'BASE'
nomeArquivo = f'SIGRH-MCZ - {nomePlanilha} - {dataReferencia}'

base = pd.read_sql(SQL, conexaoOracle)

# Criar a Planilha
planilha = gc.create(nomeArquivo)
planilha = gc.open(nomeArquivo)
pagina = municipios_arquivo.add_worksheet(nomePlanilha,rows=len(base),cols=len(base.columns))
pagina.update([base.columns.values.tolist()] + base.values.tolist())

## Salva os Dados Selecionados no SQLite

In [ ]:
%%timeit
nomeTabela = 'schema'
base.to_sql(name=nomeTabela, con=conexaoSQLite, if_exists="replace", index=False)

In [ ]:
%%timeit
pd.read_sql(f'select * from {nomeTabela}', conexaoSQLite).head()

,subsystem,tablename,columnname
0,mlve,mlveventoversao,xmlesquema
1,acer,acerto_pen_alimento,codabate
2,acer,acerto_pen_alimento,codpensao
3,acer,acerto_pen_alimento,matricula
4,acer,acerto_pen_alimento_resumo,temredutor


## Salva os Dados Selecionados em partes no SQLite

In [ ]:
%%timeit
camposSelecionados = ['orgao', 'ano', 'mes', 'folha', 'calculo', 'matricula']
tamanhoPartes=10**5
nomeTabela = 'contracheque'
for partesSQL in pd.read_sql_query(SQL, conexaoOracle, chunksize=tamanhoPartes):
    partesSQL.to_sql(name=nomeTabela, con=conexaoSQLite, if_exists="append", index=False)
    print(partesSQL[camposSelecionados].values[0])
    #print(partesSQL.iloc[0, 1])

In [ ]:
%%timeit
pd.read_sql(f'select count(*) from {nomeTabela}', conexaoSQLite)

10min 1s ± 58.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
